In [18]:
import numpy as np
import onnxruntime as ort
import cv2

In [19]:
ONNX_MODEL_PATH = "onnx_exports/anatomy_detector.onnx"
TARGET_HEIGHT = 400
TARGET_WIDTH = 600

In [ ]:
# Load ONNX model
ort_session = ort.InferenceSession(ONNX_MODEL_PATH)

# test with image 1528.png 
test_img_path = r"dataset/Dataset for Fetus Framework/Internal Test Set/Standard/1528.png"

# Preprocess the image
test_img = cv2.imread(test_img_path, cv2.IMREAD_GRAYSCALE)      # Load as grayscale
test_img = cv2.resize(test_img, (TARGET_WIDTH, TARGET_HEIGHT))  # Resize to target dimensions
test_img = ((test_img / 255.0 - 0.5) / 0.5).astype(np.float32)                # Normalize to [0, 1]
test_img = np.expand_dims(test_img, axis=0)                     # Add batch dimension
test_img = np.expand_dims(test_img, axis=0)                     # Add channel dimension

# Run inference
ort_inputs = {'input': test_img}
ort_outputs = ort_session.run(None, ort_inputs)
pred_class_conf = np.array(ort_outputs[0].squeeze(0))   # [K]
pred_boxes = np.array(ort_outputs[1].squeeze(0))        # [K, 4]

# Print results
print(f"Predicted class confidences: {pred_class_conf}")
print(f"Predicted boxes: {pred_boxes}")

Predicted class confidences: [0.8598503  0.97966266 0.99358225 0.9739133  0.9805746  0.95857215
 0.99799085 0.7284124  0.95286584]
Predicted boxes: [[ 0.43700513  0.78694963  0.1572266   0.06127717]
 [ 0.40242913  0.7212735   0.09837068  0.08814026]
 [ 0.6460256   0.8514301   0.6462361   0.09517306]
 [ 0.3362309   0.677074    0.09282832  0.19351561]
 [ 0.55376536  0.30678332  0.06022933  0.02538558]
 [ 0.5743757   0.28929353  0.07179459 -0.00681733]
 [ 0.6021862   0.28060254  0.04308435  0.03262471]
 [ 0.5827274   0.42605966  0.14385654  0.15444261]
 [ 0.36033607  0.508222    0.15419254  0.17113219]]
